In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import torch
import shutil

In [3]:
def clear_folder(path):
  for filename in os.listdir(path):
    file_path = os.path.join(path, filename)
    try:
      if os.path.isfile(file_path) or os.path.islink(file_path):
        os.unlink(file_path)
      elif os.path.isdir(file_path):
        shutil.rmtree(file_path)
    except Exception as e:
      print('Failed to delete %s. Reason: %s' % (file_path, e))

In [4]:
def create_circular_mask(h, w, center=None, radius=None):
  """
  if h % 2 == 0:
    h = h + 1
  if w % 2 == 0: 
    w = w + 1
  """
  if center is None: # use the middle of the image
    center = (int(w/2), int(h/2))
  if radius is None: # use the smallest distance between the center and image walls
    radius = min(center[0], center[1], w-center[0], h-center[1])

  Y, X = np.ogrid[:h, :w]
  dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)
  mask = dist_from_center <= radius
  return mask

In [5]:
def atleast_4d(x):
  if x.ndim < 4:
    y = np.expand_dims(np.atleast_3d(x), axis=0)
  else:
    y = x

  return y

In [6]:
physics_based_path = os.path.join('drive', 'My Drive', 'physics_based')

function **create_pipe_tensor**(PIPE_LENGTH, PIPE_RADIUS, IMG_H, IMG_W, rgb=False) returns **3D pipe mask** and **3D empty tensor**



In [7]:
def create_pipe_tensor(PIPE_LENGTH, PIPE_RADIUS, IMG_H, IMG_W, rgb=False):
  if rgb:
    pipe = np.empty((0, IMG_H, IMG_W, 3), dtype=np.float32)
  else:
    pipe = np.empty((0, IMG_H, IMG_W), dtype=np.float32)

  mask = create_circular_mask(IMG_H, IMG_W, center=None, radius=PIPE_RADIUS)

  if rgb:
    mask = atleast_4d(np.asarray(Image.fromarray(np.uint8(mask)).convert('RGB'), dtype=np.float32))
  else:
    mask = np.expand_dims(mask, axis=0)

  for i in range(PIPE_LENGTH):
    pipe = np.append(pipe, mask, axis=0)

  pipe = np.expand_dims(pipe, axis=0)
  print(f'pipe.shape: {pipe.shape}')

  """
  for xid in range(pipe.shape[0]):
    circle = pipe[xid]
    plt.imsave(os.path.join(physics_based_path, f'{xid}.png'), circle)
  """

  return torch.from_numpy(pipe)

In [8]:
PIPE_RADIUS = 48
PIPE_LENGTH = 128
IMG_W = 128
IMG_H = 128

In [10]:
clear_folder(physics_based_path)
pipe_tensor = create_pipe_tensor(PIPE_LENGTH, PIPE_RADIUS, IMG_H, IMG_W)

pipe.shape: (1, 128, 128, 128)


In [11]:
pipe_tensor.size()

torch.Size([1, 128, 128, 128])

In [12]:
pipe_tensor.size()

torch.Size([1, 128, 128, 128])

In [13]:
torch.save(pipe_tensor, os.path.join(physics_based_path, 'pipe_tensor.pt')) 

In [14]:
loaded_pipe = torch.load(os.path.join(physics_based_path, 'pipe_tensor.pt'))